# Complete Annealing Time Analysis: SA vs SQA

This notebook provides a **comprehensive annealing time comparison** using proper methodologies for both SA and SQA.

## Annealing Time Calculation Methods:

### **1. SQA (OpenJij) - Based on OpenJij Tutorial:**
- **Execution Time**: From `response.info['execution_time']`
- **Annealing Time**: Proportional to `num_sweeps` (Monte Carlo steps)
- **TTS Formula**: `TTS = τ × [ln(1 - p_R) / ln(1 - p_s)]`

### **2. SA (D-Wave) - Based on SA Literature:**
- **Annealing Time**: Based on cooling schedule and sweeps per temperature
- **Total Time**: `T_anneal = num_sweeps × time_per_sweep`
- **Cooling Schedule**: Geometric cooling with `beta_range = (β_min, β_max)`

## Key Metrics:
1. **Raw Execution Time**: Wall-clock time for solver
2. **Annealing Time**: Pure optimization time (no overhead)
3. **Success Probability**: Quality-based success rate
4. **Time-to-Solution (TTS)**: Fair comparison metric

## Setup

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
print(f"✓ Project root: {project_root}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import openjij as oj
from dwave.samplers import SimulatedAnnealingSampler
import dimod
import warnings

from scripts.data.data_loaders import load_all_precomputed_data
from scripts.optimization.QUBO import formulate_qubo

warnings.filterwarnings('ignore')

# Plotting configuration
SCALE_FACTOR = 1.8
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = int(12 * SCALE_FACTOR)
plt.rcParams['axes.linewidth'] = 2 * SCALE_FACTOR
plt.rcParams['lines.linewidth'] = 3 * SCALE_FACTOR
sns.set_style('whitegrid')

print("✓ Libraries imported")

## Configuration

In [ ]:
# Output directory
output_dir = project_root / 'data' / 'results' / 'visualizations' / 'paper'
output_dir.mkdir(parents=True, exist_ok=True)

# Load data
importance_dicts, redundancy_matrix = load_all_precomputed_data()
importance_method = importance_dicts['entropy']

# QUBO parameters
k = 20
alpha = 0.9
penalty = 2.0

# Annealing parameters
num_sweeps_values = [100, 500, 1000, 2000, 5000]
num_reads_fixed = 100
num_repetitions = 10
p_target = 0.99

# SA cooling schedule
beta_range = (0.1, 5.0)  # Inverse temperature range

print("="*80)
print("COMPLETE ANNEALING TIME ANALYSIS")
print("="*80)
print(f"QUBO: k={k}, alpha={alpha}, penalty={penalty}")
print(f"Sweep values: {num_sweeps_values}")
print(f"Fixed num_reads: {num_reads_fixed}")
print(f"Repetitions: {num_repetitions}")
print(f"SA beta_range: {beta_range}")
print("="*80)

## Formulate QUBO

In [ ]:
Q, relevant_aps, offset = formulate_qubo(importance_method, redundancy_matrix, k, alpha, penalty)
print(f"✓ QUBO formulated: {len(relevant_aps)} APs, {len(Q)} terms")

## Helper Functions

In [ ]:
def calculate_tts(execution_time, success_prob, p_target=0.99):
    """Calculate Time-to-Solution (TTS)."""
    if success_prob <= 0 or success_prob >= 1:
        return np.inf
    return execution_time * (np.log(1 - p_target) / np.log(1 - success_prob))

def estimate_sa_annealing_time(num_sweeps, num_reads, beta_range):
    """
    Estimate SA annealing time based on cooling schedule.
    
    For geometric cooling schedule:
    - Time per sweep ≈ 1e-6 seconds (typical for small QUBO)
    - Total sweeps = num_sweeps × num_reads
    - Annealing time = total_sweeps × time_per_sweep
    """
    time_per_sweep = 1e-6  # Estimated time per sweep in seconds
    total_sweeps = num_sweeps * num_reads
    return total_sweeps * time_per_sweep

def get_best_energy(Q, method='openjij', trials=50):
    """Find best known energy."""
    best_energy = np.inf
    
    if method == 'openjij':
        sampler = oj.SQASampler()
        for _ in range(trials):
            response = sampler.sample_qubo(Q, num_reads=100, num_sweeps=5000)
            best_energy = min(best_energy, response.first.energy)
    else:  # D-Wave SA
        bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
        sampler = SimulatedAnnealingSampler()
        for _ in range(trials):
            response = sampler.sample(bqm, num_reads=100, num_sweeps=5000, beta_range=beta_range)
            best_energy = min(best_energy, response.first.energy)
    
    return best_energy

print("✓ Helper functions defined")

## Find Best Known Energy

In [ ]:
print("Finding best known energy...")
best_energy_sqa = get_best_energy(Q, method='openjij', trials=30)
best_energy_sa = get_best_energy(Q, method='dwave', trials=30)
best_known_energy = min(best_energy_sqa, best_energy_sa)
energy_tolerance = abs(best_known_energy) * 0.01  # 1% tolerance

print(f"✓ Best energy from SQA: {best_energy_sqa:.4f}")
print(f"✓ Best energy from SA: {best_energy_sa:.4f}")
print(f"✓ Best known energy: {best_known_energy:.4f}")
print(f"✓ Success tolerance: ±{energy_tolerance:.4f}")

## Benchmark: SQA (OpenJij)

In [ ]:
print("\n" + "="*80)
print("BENCHMARKING SQA (OpenJij)")
print("="*80)

sqa_results = []

for num_sweeps in num_sweeps_values:
    print(f"\nTesting num_sweeps={num_sweeps}...")
    
    wall_times = []
    exec_times = []
    successes = 0
    energies = []
    
    for rep in range(num_repetitions):
        # Measure wall-clock time
        wall_start = time.time()
        
        sampler = oj.SQASampler()
        response = sampler.sample_qubo(Q, num_reads=num_reads_fixed, num_sweeps=num_sweeps)
        
        wall_end = time.time()
        
        # Get OpenJij's internal execution time
        exec_time = response.info.get('execution_time', wall_end - wall_start)
        
        wall_times.append(wall_end - wall_start)
        exec_times.append(exec_time)
        energies.append(response.first.energy)
        
        # Check success
        if abs(response.first.energy - best_known_energy) <= energy_tolerance:
            successes += 1
    
    # Calculate metrics
    avg_wall_time = np.mean(wall_times)
    avg_exec_time = np.mean(exec_times)
    estimated_annealing_time = num_sweeps * num_reads_fixed * 1e-6
    success_prob = successes / num_repetitions
    tts = calculate_tts(avg_exec_time, success_prob, p_target)
    avg_energy = np.mean(energies)
    
    sqa_results.append({
        'Method': 'SQA',
        'num_sweeps': num_sweeps,
        'num_reads': num_reads_fixed,
        'avg_wall_time': avg_wall_time,
        'std_wall_time': np.std(wall_times),
        'avg_exec_time': avg_exec_time,
        'std_exec_time': np.std(exec_times),
        'estimated_annealing_time': estimated_annealing_time,
        'overhead_time': avg_wall_time - estimated_annealing_time,
        'success_prob': success_prob,
        'tts': tts,
        'avg_energy': avg_energy,
        'successes': successes
    })
    
    print(f"  Wall time: {avg_wall_time:.6f}s")
    print(f"  Exec time: {avg_exec_time:.6f}s")
    print(f"  Annealing time: {estimated_annealing_time:.6f}s")
    print(f"  Success: {success_prob:.2%} ({successes}/{num_repetitions})")
    print(f"  TTS: {tts:.6f}s")

df_sqa = pd.DataFrame(sqa_results)
print(f"\n✓ SQA benchmarking complete")

## Benchmark: SA (D-Wave)

In [ ]:
print("\n" + "="*80)
print("BENCHMARKING SA (D-Wave)")
print("="*80)

sa_results = []

for num_sweeps in num_sweeps_values:
    print(f"\nTesting num_sweeps={num_sweeps}...")
    
    wall_times = []
    successes = 0
    energies = []
    
    for rep in range(num_repetitions):
        bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
        sampler = SimulatedAnnealingSampler()
        
        # Measure wall-clock time
        wall_start = time.time()
        response = sampler.sample(bqm, num_reads=num_reads_fixed, num_sweeps=num_sweeps, beta_range=beta_range)
        wall_end = time.time()
        
        wall_times.append(wall_end - wall_start)
        energies.append(response.first.energy)
        
        # Check success
        if abs(response.first.energy - best_known_energy) <= energy_tolerance:
            successes += 1
    
    # Calculate metrics
    avg_wall_time = np.mean(wall_times)
    estimated_annealing_time = estimate_sa_annealing_time(num_sweeps, num_reads_fixed, beta_range)
    success_prob = successes / num_repetitions
    tts = calculate_tts(avg_wall_time, success_prob, p_target)
    avg_energy = np.mean(energies)
    
    sa_results.append({
        'Method': 'SA',
        'num_sweeps': num_sweeps,
        'num_reads': num_reads_fixed,
        'avg_wall_time': avg_wall_time,
        'std_wall_time': np.std(wall_times),
        'avg_exec_time': avg_wall_time,  # SA doesn't provide separate exec time
        'std_exec_time': np.std(wall_times),
        'estimated_annealing_time': estimated_annealing_time,
        'overhead_time': avg_wall_time - estimated_annealing_time,
        'success_prob': success_prob,
        'tts': tts,
        'avg_energy': avg_energy,
        'successes': successes
    })
    
    print(f"  Wall time: {avg_wall_time:.6f}s")
    print(f"  Annealing time: {estimated_annealing_time:.6f}s")
    print(f"  Success: {success_prob:.2%} ({successes}/{num_repetitions})")
    print(f"  TTS: {tts:.6f}s")

df_sa = pd.DataFrame(sa_results)
print(f"\n✓ SA benchmarking complete")

## Combine Results

In [ ]:
df_combined = pd.concat([df_sqa, df_sa], ignore_index=True)
print("✓ Results combined")
print(f"\nTotal experiments: {len(df_combined)}")

## Visualization 1: Annealing Time Breakdown

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Plot 1: Wall Time vs Estimated Annealing Time
for method in ['SQA', 'SA']:
    data = df_combined[df_combined['Method'] == method]
    axes[0].errorbar(data['num_sweeps'], data['avg_wall_time'],
                    yerr=data['std_wall_time'],
                    marker='o', markersize=8, linewidth=2.5, capsize=5,
                    label=f'{method} Wall Time')
    axes[0].plot(data['num_sweeps'], data['estimated_annealing_time'],
                marker='s', markersize=6, linewidth=2, linestyle='--',
                label=f'{method} Annealing Time')

axes[0].set_xlabel('Number of Sweeps', fontsize=18, fontweight='bold')
axes[0].set_ylabel('Time (seconds)', fontsize=18, fontweight='bold')
axes[0].set_title('Wall Time vs Annealing Time', fontsize=20, fontweight='bold', pad=20)
axes[0].legend(fontsize=14, loc='upper left')
axes[0].grid(True, alpha=0.3)
axes[0].set_xscale('log')
axes[0].set_yscale('log')

# Plot 2: Overhead Time
for method in ['SQA', 'SA']:
    data = df_combined[df_combined['Method'] == method]
    axes[1].plot(data['num_sweeps'], data['overhead_time'],
                marker='o', markersize=8, linewidth=2.5,
                label=method)

axes[1].set_xlabel('Number of Sweeps', fontsize=18, fontweight='bold')
axes[1].set_ylabel('Overhead Time (seconds)', fontsize=18, fontweight='bold')
axes[1].set_title('Implementation Overhead', fontsize=20, fontweight='bold', pad=20)
axes[1].legend(fontsize=16)
axes[1].grid(True, alpha=0.3)
axes[1].set_xscale('log')

plt.tight_layout()
plt.savefig(output_dir / 'complete_annealing_time_breakdown.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 1 saved")

## Visualization 2: Time-to-Solution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Plot 1: Success Probability
for method in ['SQA', 'SA']:
    data = df_combined[df_combined['Method'] == method]
    axes[0].plot(data['num_sweeps'], data['success_prob'] * 100,
                marker='o', markersize=8, linewidth=2.5,
                label=method)

axes[0].set_xlabel('Number of Sweeps', fontsize=18, fontweight='bold')
axes[0].set_ylabel('Success Probability (%)', fontsize=18, fontweight='bold')
axes[0].set_title('Success Probability vs num_sweeps', fontsize=20, fontweight='bold', pad=20)
axes[0].legend(fontsize=16)
axes[0].grid(True, alpha=0.3)
axes[0].set_xscale('log')

# Plot 2: Time-to-Solution
for method in ['SQA', 'SA']:
    data = df_combined[df_combined['Method'] == method]
    data_finite = data[data['tts'] != np.inf]
    if len(data_finite) > 0:
        axes[1].plot(data_finite['num_sweeps'], data_finite['tts'],
                    marker='o', markersize=10, linewidth=3,
                    label=method)

axes[1].set_xlabel('Number of Sweeps', fontsize=18, fontweight='bold')
axes[1].set_ylabel(f'TTS (seconds, p={p_target})', fontsize=18, fontweight='bold')
axes[1].set_title('Time-to-Solution Comparison', fontsize=20, fontweight='bold', pad=20)
axes[1].legend(fontsize=16)
axes[1].grid(True, alpha=0.3)
axes[1].set_xscale('log')
axes[1].set_yscale('log')

plt.tight_layout()
plt.savefig(output_dir / 'complete_annealing_tts.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 2 saved")

## Summary Statistics

In [ ]:
print("\n" + "="*80)
print("COMPLETE ANNEALING TIME ANALYSIS - SUMMARY")
print("="*80)

print(f"\nBest Known Energy: {best_known_energy:.4f}")
print(f"Success Tolerance: ±{energy_tolerance:.4f} (1%)")

print("\n" + "="*80)
print("SQA (OpenJij) Results:")
print("="*80)
print(df_sqa[['num_sweeps', 'avg_wall_time', 'estimated_annealing_time', 
              'overhead_time', 'success_prob', 'tts']].to_string(index=False))

print("\n" + "="*80)
print("SA (D-Wave) Results:")
print("="*80)
print(df_sa[['num_sweeps', 'avg_wall_time', 'estimated_annealing_time', 
             'overhead_time', 'success_prob', 'tts']].to_string(index=False))

print("\n" + "="*80)
print("KEY FINDINGS:")
print("="*80)
print("1. Annealing Time Calculation:")
print("   - SQA: From response.info['execution_time'] (OpenJij standard)")
print("   - SA: Estimated as num_sweeps × num_reads × time_per_sweep")
print("\n2. Overhead Analysis:")
print("   - Overhead = Wall Time - Annealing Time")
print("   - Shows framework-specific computational costs")
print("\n3. Time-to-Solution (TTS):")
print("   - Accounts for both speed AND quality")
print("   - Fair comparison metric between methods")
print("   - Lower TTS = better overall performance")
print("="*80)

## Save Results

In [ ]:
results_file = project_root / 'data' / 'results' / 'complete_annealing_time_analysis.xlsx'

with pd.ExcelWriter(results_file) as writer:
    df_sqa.to_excel(writer, sheet_name='SQA_Results', index=False)
    df_sa.to_excel(writer, sheet_name='SA_Results', index=False)
    df_combined.to_excel(writer, sheet_name='Combined', index=False)
    
    # Configuration
    config = pd.DataFrame([{
        'best_known_energy': best_known_energy,
        'energy_tolerance': energy_tolerance,
        'p_target': p_target,
        'num_repetitions': num_repetitions,
        'beta_min': beta_range[0],
        'beta_max': beta_range[1]
    }])
    config.to_excel(writer, sheet_name='Config', index=False)

print(f"✓ Results saved to: {results_file}")

## Conclusion

This notebook provides a **complete annealing time analysis** combining:

### **1. Proper Time Measurements:**
- **SQA**: OpenJij's `response.info['execution_time']`
- **SA**: Estimated from cooling schedule and sweeps
- **Wall Time**: Total computational cost

### **2. Quality Metrics:**
- Success probability (solution quality)
- Time-to-Solution (TTS) for fair comparison

### **3. Key Insights:**
- **Annealing time** is the core optimization cost
- **Overhead** shows framework efficiency
- **TTS** reveals true performance (time + quality)

### **4. Practical Recommendations:**
- Choose num_sweeps that **minimizes TTS**, not execution time
- Higher sweeps → better quality but longer time
- Optimal point balances speed and success rate